<a href="https://colab.research.google.com/github/mamamoy/Electronic_sales_analytic/blob/main/Electronic_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analyzing Electronic Sales

##Exploring Data

###Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import warnings

warnings.filterwarnings("ignore")

###Importing Data

In [ ]:
df = pd.read_csv('Electronic_sales.csv')
df = df.drop(columns=df.columns[[13, 14, 15]])

###Checking data types

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Customer ID     20000 non-null  int64  
 1   Age             20000 non-null  int64  
 2   Gender          19999 non-null  object 
 3   Loyalty Member  20000 non-null  object 
 4   Product Type    20000 non-null  object 
 5   SKU             20000 non-null  object 
 6   Rating          20000 non-null  int64  
 7   Order Status    20000 non-null  object 
 8   Payment Method  20000 non-null  object 
 9   Total Price     20000 non-null  float64
 10  Unit Price      20000 non-null  float64
 11  Quantity        20000 non-null  int64  
 12  Purchase Date   20000 non-null  object 
dtypes: float64(2), int64(4), object(7)
memory usage: 2.0+ MB


###Changing the data types

In [ ]:
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])

###Checking empty/null value

In [ ]:
df[df['Gender'].isnull()]

,Customer ID,Age,Gender,Loyalty Member,Product Type,SKU,Rating,Order Status,Payment Method,Total Price,Unit Price,Quantity,Purchase Date
19999,19998,27,NaN,Yes,Laptop,LTP123,4,Completed,Bank Transfer,674.32,674.32,1,2024-01-29


In [ ]:
# 1 customer has NaN gender, so erase the row
df = df.dropna()

df.head()

In [ ]:
df.head()

,Customer ID,Age,Gender,Loyalty Member,Product Type,SKU,Rating,Order Status,Payment Method,Total Price,Unit Price,Quantity,Purchase Date
0,1000,53,Male,No,Smartphone,SKU1004,2,Cancelled,Credit Card,5538.33,791.19,7,2024-03-20
1,1000,53,Male,No,Tablet,SKU1002,3,Completed,Paypal,741.09,247.03,3,2024-04-20
2,1002,41,Male,No,Laptop,SKU1005,3,Completed,Credit Card,1855.84,463.96,4,2023-10-17
3,1002,41,Male,Yes,Smartphone,SKU1004,2,Completed,Cash,3164.76,791.19,4,2024-08-09
4,1003,75,Male,Yes,Smartphone,SKU1001,5,Completed,Cash,41.50,20.75,2,2024-05-21


###Checking duplicates

In [ ]:
df[df.duplicated()]

,Customer ID,Age,Gender,Loyalty Member,Product Type,SKU,Rating,Order Status,Payment Method,Total Price,Unit Price,Quantity,Purchase Date


###Age Grouping

In [ ]:
bins = [0, 2, 39, 59, 99]
labels = ['Baby', 'Young Adults', 'Midle-aged Adults', 'Old Adults']
df['Age Group'] = pd.cut(df['Age'], bins=bins, labels=labels, include_lowest=True)
df.head()


,Customer ID,Age,Gender,Loyalty Member,Product Type,SKU,Rating,Order Status,Payment Method,Total Price,Unit Price,Quantity,Purchase Date,Age Group
0,1000,53,Male,No,Smartphone,SKU1004,2,Cancelled,Credit Card,5538.33,791.19,7,2024-03-20,Midle-aged Adults
1,1000,53,Male,No,Tablet,SKU1002,3,Completed,Paypal,741.09,247.03,3,2024-04-20,Midle-aged Adults
2,1002,41,Male,No,Laptop,SKU1005,3,Completed,Credit Card,1855.84,463.96,4,2023-10-17,Midle-aged Adults
3,1002,41,Male,Yes,Smartphone,SKU1004,2,Completed,Cash,3164.76,791.19,4,2024-08-09,Midle-aged Adults
4,1003,75,Male,Yes,Smartphone,SKU1001,5,Completed,Cash,41.50,20.75,2,2024-05-21,Old Adults


##Visualize Data

In [ ]:
count_customer = df['Customer ID'].nunique()
print(count_customer)

12135


###Customer Identification

In [ ]:
gender_counts = df.groupby('Gender')['Customer ID'].nunique()
print(gender_counts)
print(gender_counts.sum())
fig_gender = px.pie(gender_counts, values=gender_counts, names=gender_counts.index, title='Customer by Gender')
fig_gender.show()

Gender
Female    5938
Male      6197
Name: Customer ID, dtype: int64
12135


In [ ]:
member_variation = df.groupby('Customer ID')['Loyalty Member'].nunique()
print(member_variation[member_variation > 1].value_counts())

Loyalty Member
2    2270
Name: count, dtype: int64


In [ ]:
#I assume that customer have 2 member variation on transaction is have loyalty member now
loyalty_status = df.groupby('Customer ID')['Loyalty Member'].agg(
    lambda x: 'Yes' if 'Yes' in x.values else 'No'
).reset_index()

member_counts = loyalty_status.groupby('Loyalty Member')['Customer ID'].nunique()
print(member_counts)
fig_member = px.pie(member_counts, values=member_counts, names=member_counts.index, title='Customer by Loyalty Member')
fig_member.show()

Loyalty Member
No     8302
Yes    3833
Name: Customer ID, dtype: int64


In [ ]:
age_group_counts = df.groupby('Age Group')['Customer ID'].nunique()

fig_age_group = px.pie(age_group_counts, values=age_group_counts, names=age_group_counts.index, title='Customer by Age Group')
fig_age_group.show()
age_group_counts

,Customer ID
Age Group,
Baby,0
Young Adults,4208
Midle-aged Adults,3862
Old Adults,4065


In [ ]:
grouped = df.groupby(['Customer ID', 'Gender', 'Age Group'])['Customer ID'].nunique().reset_index(name="Customer Count")
grouped['Loyalty Member Status'] = grouped['Customer ID'].map(loyalty_status.set_index('Customer ID')['Loyalty Member'])
final_grouped = grouped.groupby(['Loyalty Member Status', 'Gender', 'Age Group'])['Customer Count'].sum().reset_index()

fig = px.bar(final_grouped, x='Loyalty Member Status', y='Customer Count', color='Gender', barmode="group", facet_col="Age Group",
             title='Customer by Loyalty Member, Gender, and Age Group')
fig.show()

In [ ]:
top_3_cust_by_amount_purchase = df.groupby('Customer ID')['Total Price'].sum().nlargest(3).reset_index(name='Total Amount')
top_3_cust_by_amount_purchase['Customer ID'] = top_3_cust_by_amount_purchase['Customer ID'].astype(str)

fig = px.bar(top_3_cust_by_amount_purchase, x='Customer ID', y='Total Amount',
             title='Top 3 Customer by Amount of Purchase')
fig.show()
top_3_cust_by_amount_purchase

,Customer ID,Total Amount
0,16357,34563.70
1,16863,33035.92
2,13813,31830.16


###Quantity Purchased Performance

In [ ]:
grouped = df.groupby(['Purchase Date', 'Gender'])['Quantity'].sum().reset_index()

fig = px.line(grouped, x='Purchase Date', y='Quantity', color='Gender', title='Purchasing Quantity by Gender Over Time')
fig.show()


In [ ]:
grouped = df.groupby(['Purchase Date', 'Loyalty Member'])['Quantity'].sum().reset_index()

fig = px.line(grouped, x='Purchase Date', y='Quantity', color='Loyalty Member', title='Purchasing Quantity by Loyalty Member Over Time')
fig.show()

In [ ]:
product_type_chart = df.groupby('Product Type')['Quantity'].sum().reset_index()
product_type_chart['Percentage'] = (product_type_chart['Quantity'] / product_type_chart['Quantity'].sum()) * 100
product_type_chart = product_type_chart.sort_values(by='Percentage', ascending=False)

fig = px.bar(product_type_chart, x='Product Type', y='Quantity', color='Product Type', title='Quantity by Product Type', text=product_type_chart['Percentage'].round(2).astype(str) + '%')
fig.show()
product_type_chart

,Product Type,Quantity,Percentage
2,Smartphone,32660,29.769392
4,Tablet,22653,20.648072
3,Smartwatch,21631,19.716525
1,Laptop,21583,19.672774
0,Headphones,11183,10.193237


In [ ]:
df['Purchase Year'] = df['Purchase Date'].dt.year
product_type_by_year_chart = df.groupby(['Purchase Year', 'Product Type'])['Quantity'].sum().reset_index()
product_type_by_year_chart = product_type_by_year_chart.sort_values(by='Quantity', ascending=False)

fig = px.bar(product_type_by_year_chart, x='Purchase Year', y='Quantity', color='Product Type', barmode="group",
             title='Total Quantity of Each Year and Product Type')
fig.show()
product_type_by_year_chart

,Purchase Year,Product Type,Quantity
6,2024,Smartphone,26733
8,2024,Tablet,19464
7,2024,Smartwatch,18835
5,2024,Laptop,18783
4,2024,Headphones,11183
1,2023,Smartphone,5927
3,2023,Tablet,3189
0,2023,Laptop,2800
2,2023,Smartwatch,2796


In [ ]:
order_complete = df[df['Order Status'] == 'Completed']
sku_purchase_chart = order_complete.groupby(['Product Type', 'SKU'])['Quantity'].sum().reset_index(name='Total Purchased')
sku_purchase_chart = sku_purchase_chart.sort_values(by='Total Purchased', ascending=False)
top_3_sku = sku_purchase_chart.head(3)

fig = px.bar(top_3_sku, x='SKU', y='Total Purchased', color='Product Type', title='Top 3 SKU Purchased')
fig.show()

In [ ]:
low_3_sku = sku_purchase_chart.tail(3)

fig = px.bar(low_3_sku, x='SKU', y='Total Purchased', color='Product Type', title='Low 3 SKU Purchased')
fig.show()

In [ ]:
payment_method_chart = df.groupby(['Loyalty Member', 'Payment Method'])['Total Price'].sum().reset_index()
payment_method_chart = payment_method_chart.sort_values(by='Total Price', ascending=False)

fig = px.bar(payment_method_chart, x='Loyalty Member', y='Total Price', color='Payment Method', barmode="group", title='Purchasing by Loyalty Member and Payment Method')
fig.show()
payment_method_chart

,Loyalty Member,Payment Method,Total Price
2,No,Credit Card,14988456.85
0,No,Bank Transfer,9968921.19
4,No,PayPal,9840386.51
3,No,Debit Card,5158764.18
5,No,Paypal,5050696.98
1,No,Cash,4969525.53
8,Yes,Credit Card,3925316.07
6,Yes,Bank Transfer,2728666.58
10,Yes,PayPal,2659676.40
9,Yes,Debit Card,1577453.36


###Services

In [ ]:
order_status = df['Order Status'].value_counts()

fig_order_status = px.pie(order_status, values=order_status, names=order_status.index, title='Order Status: Canceled vs Completed')
fig_order_status.show()
order_status

,count
Order Status,
Completed,13431
Cancelled,6568


In [ ]:
dates = pd.DataFrame({'Purchase Date': df['Purchase Date'].unique()})
rating = pd.DataFrame({'Rating': df['Rating'].unique()})
join = dates.assign(key=1).merge(rating.assign(key=1), on='key').drop('key', axis=1)
grouped_ratings = df.groupby(['Purchase Date', 'Rating']).size().reset_index(name="Total")
final_result = join.merge(grouped_ratings, on=['Purchase Date', 'Rating'], how='left')
final_result['Total'] = final_result['Total'].fillna(0)
final_result = final_result.sort_values(by=['Purchase Date', 'Rating'])

fig = px.line(final_result, x='Purchase Date', y='Total', color='Rating', title='Rating Over Time')
fig.show()

In [ ]:
rating_counts = df.groupby(['Rating', 'Purchase Year'])['Rating'].value_counts().reset_index(name='Rating Count')
rating_counts = rating_counts.sort_values(by=['Rating','Purchase Year'])

fig_rating = px.histogram(rating_counts, y='Rating Count', x='Purchase Year', color='Rating', barmode='group', title='Rating Distribution')
fig_rating.show()
rating_counts

,Rating,Purchase Year,Rating Count
0,1,2024,2061
1,2,2023,529
2,2,2024,3443
3,3,2023,1617
4,3,2024,6346
5,4,2023,1
6,4,2024,2033
7,5,2023,551
8,5,2024,3418


In [ ]:
rating_counts = df.groupby('Loyalty Member')['Rating'].value_counts()
import plotly.graph_objects as go
from plotly.subplots import make_subplots

labels = rating_counts.index.get_level_values(1).unique()
values = [rating_counts.loc['Yes'].values,
          rating_counts.loc['No'].values]

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=values[0], name="Loyalty Member"), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=values[1], name="No Loyalty Member"), 1, 2)

# fig.update_layout(title_text="Rating Distribution by Loyalty Member")

fig.update_traces(hole=.4 ,hoverinfo="label+value+percent+name")
fig.update_layout(title_text="Rating Distribution by Loyalty Member",
                  annotations=[dict(text='Loyalty Member', x=sum(fig.get_subplot(1, 1).x) / 2, y=0.5,
                      font_size=20, showarrow=False, xanchor="center"),
                 dict(text='No Loyalty Member', x=sum(fig.get_subplot(1, 2).x) / 2, y=0.5,
                      font_size=20, showarrow=False, xanchor="center")])
fig.show()